In [1]:
import advent
from advent.utils import np_where_as_tuple, tadd2, tsub2
advent.scrape(2024, 20, 'csv')
data = advent.get_char_grid(20)
start, end = np_where_as_tuple(data, 'S')[0], np_where_as_tuple(data, 'E')[0]
offsets = [(0, 1), (0, -1), (1, 0), (-1, 0)]

In [2]:
def build_path(data, start, end):
    # Builds a path from start to end with the length along the path
    # The assertion statements are there because I need to know there are no splits along the path anywhere
    # Which is implied by the question I guess, but I wanted to double check
    path = {start: 0}
    current = start
    while current != end:
        c = None
        for o in offsets:
            new = tadd2(current, o)
            if data[new] != '#' and new not in path:
                assert not c, f"{start}, {end}, {current}"
                path[new] = path[current] + 1
                c = new
                continue
        assert c
        current = c
    return path

path = build_path(data, start, end)
print(path[end]) # total path is 9484 steps

9484


In [3]:
# list of cheats that save 100+ seconds
p1cheats = set([])
to_save = 100

for p in path:
    for o in offsets:
        new = tadd2(p, o)
        if data[new] != '#': continue # Can only cheat in walls
        for j in offsets:
            new2 = tadd2(new, j)
            if new2[0] < 0 or new2[0] >= data.shape[0] or new2[1] < 0 or new2[1] >= data.shape[1]: continue
            if data[new2] == '#': continue  # can only cheat out of walls
            # saving 100 seconds would be e.g. going 1, 2, c, 104: it would normally take 104, now it takes 4. so the diff has to be >= 102
            if path[new2] - path[p] >= to_save + 2: p1cheats.add((new, new2))

print(len(p1cheats))

1485


In [ ]:
# Part 2

# When I read it, I was fully convinced cheats could *only* go through walls. Meaning that I can't just
# Take all points in manhattan distance, I had to actually use BFS to find points reachable through walls
# later I found out that cheats can go through normal squares as well, and it was easy enough to fix
# But leads to quite inefficient code: I am using dijkstra to find all points reachable in 20 steps or less
# When that could literally be done with some simple math

In [4]:
# Part 2
# Considering cheats can be length 20 I want to use a BFS, but it will be a weird adj...
from typing import NamedTuple, Iterator
from advent.maze import solve_maze_no_tqdm

class Node(NamedTuple):
    length: int
    coord: tuple[int, int]

def adjacent(node: Node) -> Iterator[tuple[Node, int]]:
    if node.length == 20: return []
    # :@ This wasn't neccesarry after all!!!!
    # if data[node.coord] != '#': return [] # Cannot re-enter a cheat from a non-wall
    for o in offsets:
        new = tadd2(node.coord, o)
        if new[0] < 0 or new[0] >= data.shape[0] or new[1] < 0 or new[1] >= data.shape[1]: continue
        yield Node(node.length + 1, new), 1

#_, nodes, _ = solve_maze_no_tqdm(Node(1, (50, 81)), lambda x: False, adjacent)
#print(len([n for n in nodes if data[n.coord] != '#'])) # These are all the cheats possible from (50, 81)

In [10]:
from functools import cache
from tqdm import tqdm
p2cheats = set([])
to_save = 100

@cache
def get_cheats(wall):
    _, nodes, _ = solve_maze_no_tqdm(Node(0, wall), lambda x: False, adjacent)
    return nodes

for p in tqdm(path):
    nodes = get_cheats(p)
    for n in nodes:
        if data[n.coord] == '#': continue # cheats cannot end in walls
        if (path[n.coord] - path[p]) >= (to_save + n.length):
            p2cheats.add((p, n.coord))

# Takes like 7 minutes... obviously a lot of redundancy since we call solve_maze_no_tqdm so many times without any caching
print(len(p2cheats)) # 1027501

100%|██████████| 9485/9485 [02:51<00:00, 55.29it/s]

1027501


In [9]:
from functools import cache
from tqdm import tqdm
p2cheats = set([])
to_save = 100

@cache
def get_cheats_without_dijkstra(wall) -> list[Node]:
    result = []
    for i in range(21):
        for j in range(21-i):
            if i+j == 0: continue # That's not a cheat...
            new = tadd2(wall, (i, j))
            if new[0] < 0 or new[0] >= data.shape[0] or new[1] < 0 or new[1] >= data.shape[1]: continue
            if data[new] == '#': continue # cheats cannot end in '#'
            result.append(Node((i+j), new))
    return result

get_cheats_without_dijkstra.cache_clear()

for p in tqdm(path):
    nodes = get_cheats_without_dijkstra(p)
    for n in nodes:
        if (path[n.coord] - path[p]) >= (to_save + n.length):
            #if (p, n.coord) not in p2cheats: print((tadd2(p, (1, 1)), tadd2(n.coord, (1, 1))))
            p2cheats.add((p, n.coord))

print(len(p2cheats))

100%|██████████| 9485/9485 [00:02<00:00, 3873.16it/s]

322835


In [8]:
len(path)

9485